<h1 style=”font-size:4rem;color:red;”>RANDOM FOREST & LGBM ON SCALED DATA</h1>

- Using sclaed data the two models perform worse that unscaled data

- Also the selected features were similar to those of unscaled data

In [78]:
import pandas as pd

In [6]:
data = pd.read_csv("combined_data.csv")

In [8]:
data.head(3)

,Date,Mean-value,Interest-rate,Month,Year,Quarter,Week-of-year,Week-of-month,Day-of-week,Day-of-year,election-year
0,10/11/2016,101.28,0.69,10.0,2016.0,4.0,42.0,3.0,2.0,285.0,0
1,10/13/2016,101.30,0.66,10.0,2016.0,4.0,42.0,3.0,4.0,287.0,0
2,10/13/2016,101.30,0.66,10.0,2016.0,4.0,42.0,3.0,4.0,287.0,0


In [12]:
X = data.drop(columns = ["Date", "Mean-value", "Year"], axis=1)
y = data["Mean-value"]

In [18]:
from sklearn.preprocessing import StandardScaler

In [20]:
sclr = StandardScaler()
X_sclrd = pd.DataFrame(sclr.fit_transform(X), columns=X.columns)
X_sclrd.head(3)

,Interest-rate,Month,Quarter,Week-of-year,Week-of-month,Day-of-week,Day-of-year,election-year
0,-0.888761,0.979468,1.302228,0.971576,-0.081875,-0.721249,0.937951,-0.562159
1,-0.905330,0.979468,1.302228,0.971576,-0.081875,0.703372,0.956649,-0.562159
2,-0.905330,0.979468,1.302228,0.971576,-0.081875,0.703372,0.956649,-0.562159


In [42]:
from sklearn.model_selection import train_test_split

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_sclrd,y, test_size=0.25, random_state=42)

In [46]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1558, 8), (520, 8), (1558,), (520,))

In [48]:
from sklearn.ensemble import RandomForestRegressor

In [50]:
rf = RandomForestRegressor()

In [52]:
rf_model = rf.fit(X_train, y_train)

In [53]:
rf_model.score(X_test, y_test)

0.8873944569476913

In [56]:
from sklearn.feature_selection import RFE

In [58]:
estimator = RandomForestRegressor()
rf_selector = RFE(estimator, n_features_to_select=5, step=1)
rf_rfe = rf_selector.fit_transform(X_sclrd, y)

In [60]:
# Get the boolean mask of selected features
selected_mask = rf_selector.support_

# If X is a DataFrame, get the names of the selected features
if hasattr(X, 'columns'):
    selected_features = X_sclrd.columns[selected_mask]
else:
    # If X is a NumPy array, get the indices of selected features
    selected_features = [f"Feature_{i}" for i, selected in enumerate(selected_mask) if selected]

print("Selected Features:", selected_features)

Selected Features: Index(['Interest-rate', 'Month', 'Week-of-year', 'Day-of-year',
       'election-year'],
      dtype='object')


In [62]:
from lightgbm import LGBMRegressor

In [64]:
lgbm  = LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)

In [66]:
lgbm_model = lgbm.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000046 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 594
[LightGBM] [Info] Number of data points in the train set: 1558, number of used features: 8
[LightGBM] [Info] Start training from score 114.048588
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

In [68]:
lgbm_model.score(X_test, y_test)

0.9061042056316799

In [70]:
estimator = LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
selector = RFE(estimator, n_features_to_select=5, step=1)
lgbm_rfe = selector.fit_transform(X_sclrd, y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000179 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 593
[LightGBM] [Info] Number of data points in the train set: 2078, number of used features: 8
[LightGBM] [Info] Start training from score 114.161442
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

In [74]:
# Get the boolean mask of selected features
selected_mask = selector.support_

# If X is a DataFrame, get the names of the selected features
if hasattr(X, 'columns'):
    selected_features = X_sclrd.columns[selected_mask]
else:
    # If X is a NumPy array, get the indices of selected features
    selected_features = [f"Feature_{i}" for i, selected in enumerate(selected_mask) if selected]

print("Selected Features:", selected_features)

Selected Features: Index(['Interest-rate', 'Month', 'Week-of-year', 'Day-of-year',
       'election-year'],
      dtype='object')
